In [13]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import ndcg_score

In [14]:
# Read history and behavior files
val_size = 'demo'
behaviors_val_path = f'./files/parquet/ebnerd_{val_size}/validation/behaviors.parquet'
behaviors_val_df = pd.read_parquet(behaviors_val_path)
print(f'Raw {val_size} validation behaviors df shape:          ',behaviors_val_df.shape)


# Load the recommendation list
factorized_size = 'small'
fillnan_value = 'mean_column'
recommendations_full_lst_file_path = f'./files/pickle/recommendations_behaviors_{val_size}_factorized_{factorized_size}_fillnan_{fil lnan_value}_new.pkl'
with open(recommendations_full_lst_file_path, 'rb') as f:
   recommendations_full_lst = pickle.load(f)

Raw demo validation behaviors df shape:           (25356, 17)


In [15]:
# ----- NOT SURE IF THESE TRANSFORMATION ARE NEEDED NEITHER IF WE MAKE THE RECOMMENDATIONS IN THIS DATASET --------
behaviors_val_df = behaviors_val_df[['user_id','article_ids_inview', 'article_ids_clicked']]
behaviors_val_df = behaviors_val_df.explode('article_ids_clicked')

behaviors_val_grouped_clicked = df = behaviors_val_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index()

behaviors_val_df = behaviors_val_df.explode('article_ids_inview')

behaviors_val_grouped_inview_df = behaviors_val_df.groupby('user_id')['article_ids_inview'].apply(list).reset_index()

behaviors_val_df = pd.merge(behaviors_val_grouped_inview_df, behaviors_val_grouped_clicked, on='user_id', how='inner')

behaviors_val_df['article_ids_inview_setted_lst'] = behaviors_val_df['article_ids_inview'].apply(lambda lst: list(set(lst)))

In [16]:
print(behaviors_val_df.shape)
behaviors_val_df.head(2)

(1562, 4)


,user_id,article_ids_inview,article_ids_clicked,article_ids_inview_setted_lst
0,19181,"[9783019, 9778732, 9783122, 9783024, 9783159, ...","[9783019, 9770145, 9782519, 9782899, 9788947, ...","[8952833, 9533957, 9789446, 9786378, 9786381, ..."
1,21271,"[9784097, 7594265, 9782407, 9785267, 9784852, ...","[9785113, 9339920, 9786222, 9782407, 9784097, ...","[9785350, 9786378, 9339920, 9783824, 9784852, ..."


In [18]:
article_ids_clicked_lst = behaviors_val_df['article_ids_clicked'].tolist()
user_ids_lst = behaviors_val_df['user_id'].tolist()

In [34]:
counter, precisions, recalls, ndcgs, K = 0, [], [], [], 10



for user_id, clicked_lst, recommends_lst  in tqdm(list(zip(user_ids_lst, article_ids_clicked_lst, recommendations_full_lst))):
    
    y_true = clicked_lst
    y_pred = recommends_lst#[:K]

    precision = len(set(y_true).intersection(set(y_pred))) / len(y_pred) if len(y_pred) > 0 else 0
    recall = len(set(y_true).intersection(set(y_pred))) / len(y_true) if len(y_true) > 0 else 0

    precisions.append(precision)
    recalls.append(recall)
    #ndcgs.append(ndcg_score([y_true], [y_pred], k=K))

    if precision == 0:
        counter +=1

    print(f"User id: {user_id}, Length: {len(recommends_lst)}, Percision: {precision}, Recall: {recall}  ")

print({
    'precision@K': sum(precisions) / len(precisions)
    ,'recall@K': sum(recalls) / len(recalls)
    #,'ndcg@K': sum(ndcgs) / len(ndcgs)
    })

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_20146/3858572798.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id, clicked_lst, recommends_lst  in tqdm(list(zip(user_ids_lst, article_ids_clicked_lst, recommendations_full_lst))):


  0%|          | 0/1562 [00:00<?, ?it/s]

User id: 19181, Length: 2810, Percision: 0.0014234875444839859, Recall: 0.14814814814814814  
User id: 21271, Length: 830, Percision: 0.0, Recall: 0.0  
User id: 21774, Length: 200, Percision: 0.0, Recall: 0.0  
User id: 22779, Length: 850, Percision: 0.001176470588235294, Recall: 0.1111111111111111  
User id: 22895, Length: 2620, Percision: 0.0026717557251908397, Recall: 0.15555555555555556  
User id: 24184, Length: 2570, Percision: 0.0007782101167315176, Recall: 0.06666666666666667  
User id: 25012, Length: 870, Percision: 0.0011494252873563218, Recall: 0.09090909090909091  
User id: 25809, Length: 3290, Percision: 0.002127659574468085, Recall: 0.17073170731707318  
User id: 26594, Length: 1930, Percision: 0.0015544041450777201, Recall: 0.16666666666666666  
User id: 28327, Length: 1720, Percision: 0.0011627906976744186, Recall: 0.10526315789473684  
User id: 29497, Length: 250, Percision: 0.0, Recall: 0.0  
User id: 29562, Length: 250, Percision: 0.004, Recall: 0.2  
User id: 29681,